## Problem Statement

### Business Context

The healthcare industry is rapidly evolving, with professionals facing increasing challenges in managing vast volumes of medical data while delivering accurate and timely diagnoses. The need for quick access to comprehensive, reliable, and up-to-date medical knowledge is critical for improving patient outcomes and ensuring informed decision-making in a fast-paced environment.

Healthcare professionals often encounter information overload, struggling to sift through extensive research and data to create accurate diagnoses and treatment plans. This challenge is amplified by the need for efficiency, particularly in emergencies, where time-sensitive decisions are vital. Furthermore, access to trusted, current medical information from renowned manuals and research papers is essential for maintaining high standards of care.

To address these challenges, healthcare centers can focus on integrating systems that streamline access to medical knowledge, provide tools to support quick decision-making, and enhance efficiency. Leveraging centralized knowledge platforms and ensuring healthcare providers have continuous access to reliable resources can significantly improve patient care and operational effectiveness.

**Common Questions to Answer**

**1. Diagnostic Assistance**: "What are the common symptoms and treatments for pulmonary embolism?"

**2. Drug Information**: "Can you provide the trade names of medications used for treating hypertension?"

**3. Treatment Plans**: "What are the first-line options and alternatives for managing rheumatoid arthritis?"

**4. Specialty Knowledge**: "What are the diagnostic steps for suspected endocrine disorders?"

**5. Critical Care Protocols**: "What is the protocol for managing sepsis in a critical care unit?"

### Objective

As an AI specialist, your task is to develop a RAG-based AI solution using renowned medical manuals to address healthcare challenges. The objective is to **understand** issues like information overload, **apply** AI techniques to streamline decision-making, **analyze** its impact on diagnostics and patient outcomes, **evaluate** its potential to standardize care practices, and **create** a functional prototype demonstrating its feasibility and effectiveness.

### Data Description

The **Merck Manuals** are medical references published by the American pharmaceutical company Merck & Co., that cover a wide range of medical topics, including disorders, tests, diagnoses, and drugs. The manuals have been published since 1899, when Merck & Co. was still a subsidiary of the German company Merck.

The manual is provided as a PDF with over 4,000 pages divided into 23 sections.

## Installing and Importing Necessary Libraries and Dependencies

In [3]:
# Installation for GPU llama-cpp-python
# uncomment and run the following code in case GPU is being used
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.85 --force-reinstall --no-cache-dir -q

# Installation for CPU llama-cpp-python
# uncomment and run the following code in case GPU is not being used
# !CMAKE_ARGS="-DLLAMA_CUBLAS=off" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.85 --force-reinstall --no-cache-dir -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 42.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 154.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 254.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 293.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 259.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-community 0.0.13 requires numpy<2,>=1, but you have numpy 2.3.2 which is incompatible.
langchain 0.1.1 requires numpy<2,>=1, but you have numpy 2.3.2 which is incompatible.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 1.5.3 which is incompatible.
xarray 2025.7.1

**Note**:
- After running the above cell, kindly restart the runtime (for Google Colab) or notebook kernel (for Jupyter Notebook), and run all cells sequentially from the next cell.
- On executing the above line of code, you might see a warning regarding package dependencies. This error message can be ignored as the above code ensures that all necessary libraries and their dependencies are maintained to successfully execute the code in ***this notebook***.

In [1]:
# For installing the libraries & downloading models from HF Hub
!pip install huggingface_hub==0.23.2 pandas==1.5.3 tiktoken==0.6.0 pymupdf==1.25.1 langchain==0.1.1 langchain-community==0.0.13 chromadb==0.4.22 sentence-transformers==2.3.1 numpy==1.25.2 -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 1.5.3 which is incompatible.
xarray 2025.7.1 requires numpy>=1.26, but you have numpy 1.25.2 which is incompatible.
xarray 2025.7.1 requires packaging>=24.1, but you have packaging 23.2 which is incompatible.
xarray 2025.7.1 requires pandas>=2.2, but you have pandas 1.5.3 which is incompatible.
datasets 4.0.0 requires huggingface-hub>=0.24.0, but you have huggingface-hub 0.23.2 which is incompatible.
arviz 0.22.0 requires numpy>=1.26.0, but you have numpy 1.25.2 which is incompatible.
arviz 0.22.0 requires pandas>=2.1.0, but you have pandas 1.5.3 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.25.2 which is incompatible.
dask-cudf-cu12 25.6.0 requires pandas<2.2.4dev0,>=2.0,

In [1]:
pip install 'posthog<6.0.0'

**Note**:
- After running the above cell, kindly restart the runtime (for Google Colab) or notebook kernel (for Jupyter Notebook), and run all cells sequentially from the next cell.
- On executing the above line of code, you might see a warning regarding package dependencies. This error message can be ignored as the above code ensures that all necessary libraries and their dependencies are maintained to successfully execute the code in ***this notebook***.

In [1]:
#Libraries for downloading and loading the llm
from huggingface_hub import hf_hub_download
from llama_cpp import Llama

## Question Answering using LLM

#### Response

In [2]:
## Model configuration
#model_name_or_path = "TheBloke/Llama-2-13B-chat-GGUF"
#model_basename = "llama-2-13b-chat.Q5_K_M.gguf"

model_name_or_path = "TheBloke/Mistral-7B-Instruct-v0.2-GGUF"
model_basename = "mistral-7b-instruct-v0.2.Q6_K.gguf"


model_path = hf_hub_download(
    repo_id=model_name_or_path,
    filename=model_basename
    )

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [3]:
llm = Llama(
    model_path=model_path,
    n_threads=2,
    n_batch=256,
    n_gpu_layers=32,
    n_ctx=4096,
)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


In [4]:
def response(query,max_tokens=128,temperature=0,top_p=0.95,top_k=50):
    model_output = llm(
      prompt=query,
      max_tokens=max_tokens,
      temperature=temperature,
      top_p=top_p,
      top_k=top_k
    )

    return model_output['choices'][0]['text']

### Query 1: What is the protocol for managing sepsis in a critical care unit?

In [5]:
user_prompt = "What is the protocol for managing sepsis in a critical care unit?"
response(user_prompt)


'\n\nSepsis is a life-threatening condition that can arise from an infection, and it requires prompt recognition and aggressive management in a critical care unit. The following are general steps for managing sepsis in a critical care unit:\n\n1. Early recognition: Recognize the signs and symptoms of sepsis early and initiate treatment as soon as possible. Sepsis can present with various clinical features, including fever or hypothermia, tachycardia or bradycardia, altered mental status, respiratory distress, and lactic acidosis.\n2. Source control'

In [6]:
response(user_prompt, max_tokens=256)

Llama.generate: prefix-match hit


'\n\nSepsis is a life-threatening condition that can arise from an infection, and it requires prompt recognition and aggressive management in a critical care unit. The following are general steps for managing sepsis in a critical care unit:\n\n1. Early recognition: Recognize the signs and symptoms of sepsis early and initiate treatment as soon as possible. Sepsis can present with various clinical features, including fever or hypothermia, tachycardia or bradycardia, altered mental status, respiratory distress, and lactic acidosis.\n2. Source control: Identify and address the source of infection as quickly as possible. This may involve surgical intervention, such as drainage of an abscess or debridement of necrotic tissue.\n3. Fluid resuscitation: Administer intravenous fluids to maintain adequate blood pressure and organ perfusion. The goal is to achieve a mean arterial pressure (MAP) of at least 65 mmHg and a central venous oxygen saturation (ScvO2) of greater than 70%.\n4. Antibiotics

### Query 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

In [7]:
user_prompt2 = "What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?"
response(user_prompt2)

Llama.generate: prefix-match hit


'\n\nAppendicitis is a medical condition characterized by inflammation of the appendix, a small pouch-like structure that extends from the large intestine. The symptoms of appendicitis can vary from person to person, but some common signs include:\n\n1. Abdominal pain: The pain is typically located in the lower right side of the abdomen and may be constant or come and go. It may start as a mild discomfort that worsens over time.\n2. Loss of appetite: People with appendicitis often lose their appetite due to abdominal pain and nausea'

### Query 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

In [8]:
user_prompt3 = "What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?"
response(user_prompt3)

Llama.generate: prefix-match hit


"\n\nSudden patchy hair loss, also known as alopecia areata, is a common autoimmune disorder that affects the hair follicles. It can result in round or oval bald patches on the scalp, but it can also occur on other parts of the body such as the beard area, eyebrows, and eyelashes.\n\nThe exact cause of alopecia areata is not known, but it's believed to be related to a problem with the immune system. Some possible triggers for this condition include stress, genetics, viral infections, and certain medications."

In [9]:
response(user_prompt3, temperature=0.9)

Llama.generate: prefix-match hit


'\n\nSudden patchy hair loss, also known as alopecia areata, is a common autoimmune disorder that affects the hair follicles, leading to hair loss in small patches on the scalp or other parts of the body. The exact cause of alopecia areata is not fully understood, but it is believed to be related to a dysfunction in the immune system.\n\nThere is currently no cure for alopecia areata, but there are various treatments that can help promote hair regrowth and prevent further hair loss:\n\n1. Topical corticosteroids'

### Query 4:  What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [10]:
user_prompt4 = "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?"
response4 = response(user_prompt4)
print(response4)

Llama.generate: prefix-match hit




A person who has sustained a physical injury to the brain tissue may require various treatments depending on the severity and location of the injury. Here are some common treatments that may be recommended:

1. Emergency care: In case of a traumatic brain injury (TBI), it is essential to seek emergency medical attention as soon as possible. The primary goal of emergency care is to prevent further damage to the brain, stabilize vital signs, and manage any life-threatening conditions.
2. Medications: Depending on the symptoms, healthcare professionals may prescribe medications to manage various conditions associated with a


In [11]:
user_prompt4 = "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?"
response4 = response(user_prompt4, max_tokens=256)

Llama.generate: prefix-match hit


### Query 5: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

In [12]:
user_prompt5 = "What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?"
response5 = response(user_prompt5)
print(response5)

Llama.generate: prefix-match hit




First and foremost, if you suspect that someone has fractured their leg while hiking, it's essential to ensure their safety and prevent further injury. Here are some necessary precautions:

1. Keep the person calm and still: Encourage them to remain as still as possible to minimize pain and prevent worsening the injury.
2. Assess the situation: Check for any signs of shock, such as pale skin, rapid heartbeat, or shallow breathing. If you notice these symptoms, seek medical help immediately.
3. Immobilize the leg: Use a splint, sl


## Question Answering using LLM with Prompt Engineering

In [13]:
# function to generate, process, and return the response from the LLM
def generate_llama_response(user_prompt, max_tokens=128,temperature=0,top_p=0.95,top_k=50, repeat_penalty=1.2):

    # System message
    system_message = """
    [INST]<<SYS>> Respond to the user question based on the user prompt. Keep the answers precise backed with medical facts. Focus on the medical condition asked and avoid generic terms.<</SYS>>[/INST]
    """

    # Combine user_prompt and system_message to create the prompt
    prompt = f"{user_prompt}\n{system_message}"

    # Generate a response from the LLaMA model
    response = llm(
        prompt=prompt,
        max_tokens=max_tokens,
        temperature=temperature,
        top_p=top_p,
        repeat_penalty=repeat_penalty,
        top_k=top_k,
        stop=['INST'],
        echo=False
    )

    # Extract and return the response text
    response_text = response["choices"][0]["text"]
    return response_text

### Query 1: What is the protocol for managing sepsis in a critical care unit?

In [14]:
user_prompt = "What is the protocol for managing sepsis in a critical care unit?"
response = generate_llama_response(user_prompt, max_tokens=512)
print(response)

Llama.generate: prefix-match hit


1. Recognition: Early recognition of sepsis is crucial. Look for signs of infection, organ dysfunction (using Sequential [Sepsis-related] Organ Failure Assessment or SOFA score), and suspected infection in the medical history.

2. Resuscitation: Initiate fluid resuscitation to maintain adequate tissue perfusion. Aim for a mean arterial pressure ≥65 mmHg and central venous oxygen saturation >70%.

3. Antibiotics: Administer broad-spectrum antibiotics as soon as possible based on the suspected infection source, culture results if available, and local guidelines.

4. Source control: Address the underlying cause of sepsis by performing appropriate interventions such as drainage of abscesses or debridement of necrotic tissue.

5. Vasopressors: If fluid resuscitation fails to maintain adequate blood pressure, administer vasopressors to support circulation.

6. Corticosteroids: Consider using corticosteroids in cases of septic shock that do not respond to initial therapy within 1 hour.

7. In

### Query 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

In [15]:
user_prompt2 = "What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?"
response2 = generate_llama_response(user_prompt2, max_tokens=128)
print(response2)

Llama.generate: prefix-match hit


 Appendicitis is a common inflammatory condition of the appendix, a small pouch that extends from the cecum (the beginning of the large intestine). The following are some common symptoms for appendicitis:

1. Abdominal pain: This is usually located in the lower right side of the abdomen and may be constant or come and go. It can feel sharp, dull, or crampy and worsens with movement or pressure on the area.
2. Loss of appetite: People with appendicitis often lose their appetite due to abdominal pain and discomfort


### Query 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

In [16]:
user_prompt3 = "What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?"
response3 = generate_llama_response(user_prompt3,temperature=0.9)
print(response3)

Llama.generate: prefix-match hit


There are several potential causes for sudden, patchy hair loss. Some common conditions that may lead to localized bald spots include:

1. Alopecia Areata: This is an autoimmune disease where your immune system attacks the hair follicles, causing hair loss in round patches on the scalp, beard, or other areas of the body. According to Mayo Clinic, about 6 million people in the United States have alopecia areata.
2. Tinea Capitis: Also known as ringworm of the scalp, it is a fungal infection that can cause hair


In [17]:
response3 = generate_llama_response(user_prompt3, max_tokens=512)
print(response3)

Llama.generate: prefix-match hit


1. Causes of Sudden Patchy Hair Loss:
       a) Alopecia Areata: This is an autoimmune disorder where your immune system attacks hair follicles, leading to sudden hair loss in small patches on the scalp or other areas of the body. It can affect people of all ages and ethnicities.
       b) Trauma or Stress: Physical trauma, emotional stress, or surgery may cause temporary hair loss known as telogen effluvium. This condition results from an increased number of hairs entering the resting phase (telogen), which leads to shedding.
       c) Fungal Infections: Scalp infections caused by fungi like ringworm can lead to patchy hair loss, redness, itching, and scaling on the scalp.
       d) Nutritional Deficiencies: Lack of essential nutrients such as iron, zinc, or vitamin B12 may cause sudden hair loss in patches.
    2. Treatments for Sudden Patchy Hair Loss:
       a) Alopecia Areata: Topical treatments like minoxidil and corticosteroids can help stimulate hair growth. In severe cases, sy

### Query 4:  What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [18]:
user_prompt4 = "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?"
response4 = generate_llama_response(user_prompt4, max_tokens=256)
print(response4)

Llama.generate: prefix-match hit


For a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function, also known as a traumatic brain injury (TBI), treatment may include:

1. Emergency care: This includes managing airway and breathing, controlling bleeding, preventing infection, and addressing any life-threatening conditions.
2. Medications: Depending on the specific symptoms of the TBI, medications may be prescribed to manage pain, reduce swelling or inflammation in the brain, prevent seizures, or control other medical conditions.
3. Surgery: In some cases, surgery may be necessary to remove blood clots or repair skull fractures that could worsen brain damage or lead to further complications.
4. Rehabilitation: This includes physical therapy, occupational therapy, speech and language therapy, cognitive rehabilitation, and social support services to help the person regain lost skills and adapt to any permanent impairments.
5. Assistive devices: Depending o

### Query 5: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

In [19]:
user_prompt5 = "What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?"
response5 = generate_llama_response(user_prompt5)
print(response5)

Llama.generate: prefix-match hit


1. Assessing the Fracture: The first step is to assess the severity of the fracture. This can be done by checking for signs of deformity, swelling, bruising, numbness or tingling sensations, and if there is open skin or bone protruding from the wound.
    2. Immobilizing the Leg: Use a splint or a makeshift sling to immobilize the leg to prevent further damage and reduce pain. If possible, elevate the injured leg above heart level to minimize swelling.
    3. Controlling Bleeding


In [20]:
response5 = generate_llama_response(user_prompt5, max_tokens=1024)
print(response5)

Llama.generate: prefix-match hit


1. Assessing the Fracture: The first step is to assess the severity of the fracture. This can be done by checking for signs of deformity, swelling, bruising, numbness or tingling sensations, and if there's open skin or bone protruding. If the leg appears misshapen or the person is unable to bear weight on it, these could indicate a more serious fracture that requires immediate medical attention.
    2. Immobilizing the Fractured Leg: Use a splint, sling, or a makeshift immobilizer (such as a hiking pole) to prevent any further movement of the leg and reduce pain. Be careful not to apply too much pressure on the injury site that could restrict blood flow.
    3. Provide Comfort and Support: Keep the person warm by covering them with blankets or insulating materials, and provide fluids to maintain hydration. Encourage them to rest as much as possible to conserve energy for their journey back to civilization.
    4. Seek Medical Help: If the fracture is severe, it's essential to seek prof

In [21]:
# function to generate, process, and return the response from the LLMwith different system prompt
def generate_llama_response(user_prompt):

    # System message
    system_message = """
    [INST]<<SYS>> Respond to the user question based on the user prompt. Keep the answer short focusing on context of the question.<</SYS>>[/INST]
    """

    # Combine user_prompt and system_message to create the prompt
    prompt = f"{user_prompt}\n{system_message}"

    # Generate a response from the LLaMA model
    response = llm(
        prompt=prompt,
        max_tokens=1024,
        temperature=0.01,
        top_p=0.95,
        repeat_penalty=1.2,
        top_k=50,
        stop=['INST'],
        echo=False
    )

    # Extract and return the response text
    response_text = response["choices"][0]["text"]
    return response_text

In [22]:
user_prompt = "What is the protocol for managing sepsis in a critical care unit?"
response = generate_llama_response(user_prompt)
print(response)

Llama.generate: prefix-match hit


1. Recognize and suspect sepsis early using clinical suspicion, laboratory findings (elevated white blood cell count, lactate), and scoring systems like Sequential Organ Failure Assessment (SOFA) or Quick Sequential [Sepsis-related Organ Failure Assessment (qSOFA).](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3671285/)
    2. Initiate early goal-directed therapy: administer broad-spectrum antibiotics, fluid resuscitation to maintain adequate tissue perfusion (MAP > 65 mmHg and CVP or central venous pressure > 8 cm H2O), and consider vasopressors if needed for mean arterial pressure support.
    3. Monitor vital signs closely: assess respiratory status, cardiac output, urine output, electrolytes, lactate levels, and organ function regularly.
    4. Administer corticosteroids in specific situations (septic shock unresponsive to fluid resuscitation).
    5. Consider source control for infection if possible: drain abscesses or perform surgical interventions as needed.
    6. Provide suppo

In [23]:
user_prompt2 = "What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?"
response2 = generate_llama_response(user_prompt2)
print(response2)

Llama.generate: prefix-match hit


Appendicitis is typically characterized by symptoms such as:
    - Sudden pain in the lower right abdomen, often starting around the navel and moving downward
    - Loss of appetite
    - Nausea and vomiting
    - Fever
    - Abdominal swelling
    
Appendicitis cannot be cured with medicine alone. If left untreated, it can lead to rupture of the appendix, which is a medical emergency. The standard treatment for appendicitis is surgical removal of the appendix through an operation called appendectomy. This procedure helps prevent complications and reduces the risk of recurrence.


In [24]:
user_prompt3 = "What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?"
response3 = generate_llama_response(user_prompt3)
print(response3)

Llama.generate: prefix-match hit


1. Possible Causes: Sudden patchy hair loss can have various reasons such as alopecia areata (an autoimmune condition), nutritional deficiencies, stress, or certain medications. Infections like ringworm can also cause bald spots.

2. Treatments: For localized bald patches due to alopecia areata, topical treatments like minoxidil and corticosteroids may be prescribed by a healthcare professional. Other options include immunotherapy or hair transplantation in severe cases. Proper nutrition, stress management techniques, and avoiding triggering medications can help prevent patchy hair loss caused by other reasons.

3. Prevention: Maintaining good overall health through balanced diet, regular exercise, adequate sleep, and managing stress effectively are essential for healthy hair growth. Avoid using harsh chemicals on your scalp or pulling out hairs excessively to minimize the risk of sudden bald patches.


In [25]:
user_prompt4 = "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?"
response4 = generate_llama_response(user_prompt4)
print(response4)

Llama.generate: prefix-match hit


1. Immediate medical attention: The person should be taken to a hospital for an evaluation and treatment of any life-threatening injuries or conditions.
2. Rehabilitation therapy: Depending on the extent and location of brain damage, various therapies may be recommended such as physical therapy, occupational therapy, speech therapy, cognitive rehabilitation, and behavioral modification to help restore function and improve quality of life.
3. Medications: Prescription medications may be used to manage symptoms related to brain injury like pain, seizures, depression, or anxiety.
4. Surgery: In some cases, surgery might be necessary to remove blood clots, repair skull fractures, or alleviate pressure on the brain caused by swelling.
5. Assistive devices and technology: Devices such as wheelchairs, walkers, communication aids, and assistive technologies can help individuals with brain injuries regain independence and improve their daily functioning.
6. Support groups: Joining support group

In [26]:
user_prompt5 = "What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?"
response5 = generate_llama_response(user_prompt5)
print(response5)

Llama.generate: prefix-match hit


1. Assess the severity: Check if it's an open or closed fracture, and if there is any swelling, bruising, deformity, or numbness below the injury site.
    2. Immobilize the leg: Use a splint or sling to prevent movement and further damage. If possible, elevate the injured leg above heart level to reduce swelling.
    3. Seek medical help: Call emergency services if you're in an isolated area or unable to transport the person yourself. In non-emergency situations, schedule a doctor’s appointment as soon as possible.
    4. Pain management: Provide over-the-counter pain medication like acetaminophen or ibuprofen, following dosage instructions carefully.
    5. Proper nutrition and hydration: Ensure the person is getting enough nutrients to support bone healing and adequate fluids for proper hydration.
    6. Follow doctor's advice: Adhere to any specific care instructions provided by your healthcare professional, such as physical therapy exercises or use of a cast or brace.
    7. Monit

## Data Preparation for RAG

In [27]:
#Libraries for processing dataframes,text
import json,os
import tiktoken
import pandas as pd

#Libraries for Loading Data, Chunking, Embedding, and Vector Databases
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain_community.vectorstores import Chroma

### Loading the Data

In [28]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [29]:
medical_diagnosis_pdf_path = "/content/drive/MyDrive/Colab Notebooks/Projects/NLP/medical_diagnosis_manual.pdf"

In [30]:
pdf_loader = PyMuPDFLoader(medical_diagnosis_pdf_path)

In [31]:
medical_diagnosis_manual = pdf_loader.load()

### Data Overview

#### Checking the first 5 pages

In [32]:
for i in range(5):
    print(f"Page Number : {i+1}",end="\n")
    print(medical_diagnosis_manual[i].page_content,end="\n")

Page Number : 1
neha.artica@gmail.com
43A5MYC1TD
eant for personal use by neha.artica@gm
shing the contents in part or full is liable 

Page Number : 2
neha.artica@gmail.com
43A5MYC1TD
This file is meant for personal use by neha.artica@gmail.com only.
Sharing or publishing the contents in part or full is liable for legal action.

Page Number : 3
Table of Contents
1
Front    ................................................................................................................................................................................................................
1
Cover    .......................................................................................................................................................................................................
2
Front Matter    .........................................................................................................................................................................................

#### Checking the number of pages

In [33]:
len(medical_diagnosis_manual)

4114

### Data Chunking

In [34]:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    encoding_name='cl100k_base',
    chunk_size=512,
    chunk_overlap= 20
)

In [35]:
document_chunks = pdf_loader.load_and_split(text_splitter)

In [36]:
len(document_chunks)

8463

In [37]:
document_chunks[0].page_content

'neha.artica@gmail.com\n43A5MYC1TD\neant for personal use by neha.artica@gm\nshing the contents in part or full is liable'

In [38]:
document_chunks[-1].page_content

"Z\nZafirlukast 1879\nZalcitabine 1451\nin children 2854\nZaleplon 1709\nZanamivir 1407\nin influenza 1407, 1929\nZAP-70 (zeta-associated protein 70) deficiency 1092, 1108\nZavanelli maneuver 2680\nZellweger syndrome 2383, 3023\nZenker's diverticulum 125\nZidovudine 1451, 1453\nin children 2854\nZileuton 1881\nin asthma 1880\nZinc 49, 55, 3431-3432\nin common cold 1405\ndeficiency of 11, 49, 55\nin dermatophytoses 705\npoisoning with 3328, 3353\nrecommended dietary allowances for 50\nreference values for 3499\ntoxicity of 49, 55\ncopper deficiency and 49\nin Wilson's disease 52\nZinc oxide 2233\ngelatin formulation of 646, 672\nZinc pyrithione 647\nZinc shakes 55\nZipper injury 3239, 3240\nZiprasidone\nin agitation 1492\nin bipolar disorder 3059\npoisoning with 3347\nin schizophrenia 1566\nZoledronate 359, 361, 848\nZollinger-Ellison syndrome 95, 199, 200-201, 910\nmastocytosis vs 1125\nMenetrier's disease vs 132\npeptic ulcer disease vs 134\nZolmitriptan 1721\nZolpidem 1709, 3103\nZon

### Embedding

In [39]:
embedding_model = SentenceTransformerEmbeddings(model_name='thenlper/gte-large')

In [40]:
embedding_1 = embedding_model.embed_query(document_chunks[0].page_content)

In [41]:
len(embedding_1)

1024

In [42]:
print(embedding_1)

[-0.029329389333724976, 0.018194282427430153, -0.0033010593615472317, -0.0026498103979974985, -0.0032665091566741467, -0.05095609650015831, 0.01653674617409706, 0.0320604145526886, -0.006114642135798931, 0.009387011639773846, 0.02607172355055809, 0.012917481362819672, 0.021929845213890076, -0.04317786917090416, -0.014668034389615059, 0.006613133009523153, -0.0031888573430478573, -0.03750202804803848, -0.006564529146999121, 0.016960712149739265, 0.02485170215368271, 0.012375133112072945, -0.08553222566843033, -0.02859608456492424, -0.015356861986219883, 0.021128837019205093, 0.04233521968126297, 0.0016557095805183053, 0.05116812884807587, 0.0223923921585083, -0.028187192976474762, -0.0014164346503093839, 0.026658834889531136, -0.04196138679981232, -0.005035104230046272, -0.001956908032298088, 0.044417139142751694, -0.02783725969493389, -0.02982799895107746, -0.04598867520689964, 0.005317986942827702, -0.008231248706579208, 0.03848383203148842, -0.019536778330802917, -0.01428713742643594

### Vector Database

In [43]:
out_dir = 'medical_diagnosis_db'

if not os.path.exists(out_dir):
  os.makedirs(out_dir)

In [44]:
vectorstore = Chroma.from_documents(
    document_chunks,
    embedding_model,
    persist_directory=out_dir
)

In [45]:
vectorstore = Chroma(persist_directory=out_dir,embedding_function=embedding_model)

In [46]:
vectorstore.embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False})
  (2): Normalize()
), model_name='thenlper/gte-large', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False)

### Retriever

In [47]:
retriever = vectorstore.as_retriever(
    search_type='similarity',
    search_kwargs={'k': 2}
)

In [48]:
rel_docs = retriever.get_relevant_documents(query= 'What is the protocol for managing sepsis in a critical care unit?')
rel_docs

[Document(page_content="16 - Critical Care Medicine\nChapter 222. Approach to the Critically Ill Patient\nIntroduction\nCritical care medicine specializes in caring for the most seriously ill patients. These patients are best\ntreated in an ICU staffed by experienced personnel. Some hospitals maintain separate units for special\npopulations (eg, cardiac, surgical, neurologic, pediatric, or neonatal patients). ICUs have a high\nnurse:patient ratio to provide the necessary high intensity of service, including treatment and monitoring\nof physiologic parameters.\nSupportive care for the ICU patient includes provision of adequate nutrition (see p. 21) and prevention of\ninfection, stress ulcers and gastritis (see p. 131), and pulmonary embolism (see p. 1920). Because 15 to\n25% of patients admitted to ICUs die there, physicians should know how to minimize suffering and help\ndying patients maintain dignity (see p. 3480).\nPatient Monitoring and Testing\nSome monitoring is manual (ie, by di

### System and User Prompt Template

1. The system message describing the assistant's role.
2. A user message template including context and the question.

In [49]:
qna_system_message = """
You are an medical assistant whose work is to review the report and provide the appropriate answers from the context.
User input will have the context required by you to answer user questions.
This context will begin with the token: ###Context.
The context contains references to specific portions of a document relevant to the user query.

User questions will begin with the token: ###Question.

Please answer only using the context provided in the input. Do not mention anything about the context in your final answer.

If the answer is not found in the context, respond "I don't know".
"""

In [50]:
qna_user_message_template = """
###Context
Here are some documents that are relevant to the question mentioned below.
{context}

###Question
{question}
"""

### Response Function

In [51]:
def generate_rag_response(user_input,k=3,max_tokens=128,temperature=0,top_p=0.95,top_k=50):
    global qna_system_message,qna_user_message_template
    # Retrieve relevant document chunks
    relevant_document_chunks = retriever.get_relevant_documents(query=user_input,k=k)
    context_list = [d.page_content for d in relevant_document_chunks]

    # Combine document chunks into a single context
    context_for_query = ". ".join(context_list)

    user_message = qna_user_message_template.replace('{context}', context_for_query)
    user_message = user_message.replace('{question}', user_input)

    prompt = qna_system_message + '\n' + user_message

    # Generate the response
    try:
        response = llm(
                  prompt=prompt,
                  max_tokens=max_tokens,
                  temperature=temperature,
                  top_p=top_p,
                  top_k=top_k
                  )

        # Extract and print the model's response
        response = response['choices'][0]['text'].strip()
    except Exception as e:
        response = f'Sorry, I encountered the following error: \n {e}'

    return response

## Question Answering using RAG



```
# This is formatted as code
```

### Query 1: What is the protocol for managing sepsis in a critical care unit?

In [52]:
user_input = "What is the protocol for managing sepsis in a critical care unit?"
print(generate_rag_response(user_input))

Llama.generate: prefix-match hit


I don't know. The context does not provide information on the protocol for managing sepsis in a critical care unit.


### Query 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

In [53]:
user_input2 = "What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?"
print(generate_rag_response(user_input2))

Llama.generate: prefix-match hit


The common symptoms for appendicitis include pain in the lower right abdomen, loss of appetite, nausea or vomiting, fever, and a feeling of being sick. Appendicitis cannot be cured via medicine alone as it requires surgical removal of the appendix. The treatment for appendicitis is open or laparoscopic appendectomy.


### Query 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

In [54]:
user_input3 = "What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?"
print(generate_rag_response(user_input3, max_tokens=512))

Llama.generate: prefix-match hit


Based on the context provided, the condition being described is Alopecia Areata. The context mentions that it is a sudden patchy hair loss in people with no obvious skin or systemic disorder. It also states that alopecia areata is thought to be an autoimmune disorder affecting genetically susceptible people exposed to unclear environmental triggers.

The effective treatments or solutions for addressing sudden patchy hair loss, as mentioned in the context, include corticosteroids, retinoids, or immunosuppressants. These medications are used to help reduce inflammation and suppress the immune system's attack on the hair follicles. However, it is important to note that these treatments may not work for everyone and their effectiveness can vary from person to person.

Therefore, the answer to the user question would be: The effective treatments or solutions for addressing sudden patchy hair loss include corticosteroids, retinoids, or immunosuppressants. These medications are used to help r

### Query 4:  What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [55]:
user_input4 = "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?"
print(generate_rag_response(user_input4, max_tokens=512, temperature=0.9))

Llama.generate: prefix-match hit


I. Initial treatment includes ensuring a reliable airway, maintaining adequate ventilation, oxygenation, and blood pressure, as well as surgery if needed for monitoring intracranial pressure, decompression, or removal of hematomas. In the first few days after the injury, maintaining adequate brain perfusion and oxygenation and preventing complications of altered sensorium are important. Subsequently, many patients require rehabilitation.
II. In the US, TBI is a common cause of death and disability, with causes including motor vehicle crashes, falls, assaults, and sports activities. The structural changes from head injury may be gross or microscopic, depending on the mechanism and forces involved. Clinical manifestations vary in severity and consequences. Injuries are commonly categorized as open or closed.
III. Open injuries involve penetration of the scalp and skull (and usually the meninges and underlying brain tissue) and typically involve bullets or sharp objects, but a skull fract

### Query 5: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

In [56]:
user_input5 = "What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?"
print(generate_rag_response(user_input5))

Llama.generate: prefix-match hit


Based on the context provided, here's the answer:
1. Elevate the injured limb above heart level for the first 2 days to minimize swelling.
2. After 48 hours, apply heat (using a heating pad) for 15 to 20 minutes to relieve pain and speed healing.
3. Immobilize the joints proximal and distal to the injury using elastic bandages or a cast, depending on the severity of the fracture and the length of immobilization required.
4. For casted patients: keep the cast dry


### Fine-tuning

### Query 1: What is the protocol for managing sepsis in a critical care unit?

In [57]:
user_input = "What is the protocol for managing sepsis in a critical care unit?"
generate_rag_response(user_input, top_p=0.98, top_k=20, max_tokens=256)

Llama.generate: prefix-match hit


"I don't know. The context does not provide information on the protocol for managing sepsis in a critical care unit."

In [59]:
user_input = "What is the protocol for managing sepsis in a critical care unit?"
generate_rag_response(user_input, top_p=0.98, top_k=20,temperature=0.9, max_tokens=256)

Llama.generate: prefix-match hit


"I don't know. The context provided does not discuss the specific protocol for managing sepsis in a critical care unit."

### Query 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

In [58]:
user_input2 = "What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?"
generate_rag_response(user_input2, top_p=0.90, top_k=50, max_tokens=256)

Llama.generate: prefix-match hit


'The common symptoms for appendicitis include pain in the lower right abdomen, loss of appetite, nausea or vomiting, fever, and a feeling of being sick. Appendicitis cannot be cured via medicine alone as it requires surgical removal of the appendix. The treatment for appendicitis is open or laparoscopic appendectomy.'

### Query 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

In [60]:
user_input3 = "What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?"
generate_rag_response(user_input3, top_p=0.98, top_k=20, max_tokens=512)

Llama.generate: prefix-match hit


"Based on the context provided, the condition being described is Alopecia Areata. The context mentions that it is a sudden patchy hair loss in people with no obvious skin or systemic disorder. It also states that alopecia areata is thought to be an autoimmune disorder affecting genetically susceptible people exposed to unclear environmental triggers.\n\nThe effective treatments or solutions for addressing sudden patchy hair loss, as mentioned in the context, include corticosteroids, retinoids, or immunosuppressants. These medications are used to help reduce inflammation and suppress the immune system's attack on the hair follicles. However, it is important to note that these treatments may not work for everyone and their effectiveness can vary from person to person.\n\nTherefore, the answer to the user question would be: The effective treatments or solutions for addressing sudden patchy hair loss include corticosteroids, retinoids, or immunosuppressants. These medications are used to h

### Query 4:  What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [61]:
user_input4 = "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?"
generate_rag_response(user_input4, top_p=0.98, top_k=20, max_tokens=1024)

Llama.generate: prefix-match hit


'Answer: The initial treatment for a person with traumatic brain injury (TBI) includes ensuring a reliable airway and maintaining adequate ventilation, oxygenation, and blood pressure. Surgery may be needed in patients with more severe injuries to place monitors to track and treat intracranial pressure, decompress the brain if intracranial pressure is increased, or remove intracranial hematomas. In the first few days after the injury, maintaining adequate brain perfusion and oxygenation and preventing complications of altered sensorium are important. Subsequently, many patients require rehabilitation.'

### Query 5: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

In [62]:
user_input5 = "What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?"
generate_rag_response(user_input5, max_tokens=256)

Llama.generate: prefix-match hit


"Based on the context provided, here's the answer:\n1. Elevate the injured limb above heart level for the first 2 days to minimize swelling.\n2. After 48 hours, apply heat (using a heating pad) for 15 to 20 minutes to relieve pain and speed healing.\n3. Immobilize the joints proximal and distal to the injury using elastic bandages or a cast, depending on the severity of the fracture and the length of immobilization required.\n4. For casted patients: keep the cast dry, never put an object inside it, inspect the edges and skin around the cast daily, and pad any rough edges with soft material to prevent irritation.\n5. If severe swelling is likely, a cast may be bivalved (cut open) to allow for drainage and reduce pressure.\n6. Follow-up care and rehabilitation as recommended by a healthcare professional."

## Output Evaluation

Let us now use the LLM-as-a-judge method to check the quality of the RAG system on two parameters - retrieval and generation. We illustrate this evaluation based on the answeres generated to the question from the previous section.

- We are using the same Mistral model for evaluation, so basically here the llm is rating itself on how well he has performed in the task.

In [64]:
groundedness_rater_system_message  =  """
You are tasked with rating AI generated answers to medical questions posed by users.
You will be presented a question, context used by the AI system to generate the answer and an AI generated answer to the question.
In the input, the question will begin with ###Question, the context will begin with ###Context while the AI generated answer will begin with ###Answer.

Evaluation criteria:
The task is to judge the extent to which the metric is followed by the answer.
1 - The metric is not followed at all
2 - The metric is followed only to a limited extent
3 - The metric is followed to a good extent
4 - The metric is followed mostly
5 - The metric is followed completely

Metric:
The answer should be derived only from the information presented in the context

Instructions:
1. First write down the steps that are needed to evaluate the answer as per the metric.
2. Give a step-by-step explanation if the answer adheres to the metric considering the question and context as the input.
3. Next, evaluate the extent to which the metric is followed.
4. Use the previous information to rate the answer using the evaluaton criteria and assign a score.
"""

In [65]:
relevance_rater_system_message = """
You are tasked with rating AI generated answers to questions posed by users.
You will be presented a question, context used by the AI system to generate the answer and an AI generated answer to the question.
In the input, the question will begin with ###Question, the context will begin with ###Context while the AI generated answer will begin with ###Answer.

Evaluation criteria:
The task is to judge the extent to which the metric is followed by the answer.
1 - The metric is not followed at all
2 - The metric is followed only to a limited extent
3 - The metric is followed to a good extent
4 - The metric is followed mostly
5 - The metric is followed completely

Metric:
Relevance measures how well the answer addresses the main aspects of the question, based on the context.
Consider whether all and only the important aspects are contained in the answer when evaluating relevance.

Instructions:
1. First write down the steps that are needed to evaluate the context as per the metric.
2. Give a step-by-step explanation if the context adheres to the metric considering the question as the input.
3. Next, evaluate the extent to which the metric is followed.
4. Use the previous information to rate the context using the evaluaton criteria and assign a score.
"""

In [66]:
user_message_template = """
###Question
{question}

###Context
{context}

###Answer
{answer}
"""

In [73]:
def generate_ground_relevance_response(user_input,k=3,max_tokens=512,temperature=0,top_p=0.95,top_k=50):
    global qna_system_message,qna_user_message_template
    # Retrieve relevant document chunks
    relevant_document_chunks = retriever.get_relevant_documents(query=user_input,k=3)
    context_list = [d.page_content for d in relevant_document_chunks]
    context_for_query = ". ".join(context_list)

    # Combine user_prompt and system_message to create the prompt
    prompt = f"""[INST]{qna_system_message}\n
                {'user'}: {qna_user_message_template.format(context=context_for_query, question=user_input)}
                [/INST]"""

    response = llm(
            prompt=prompt,
            max_tokens=max_tokens,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            stop=['INST'],
            )

    answer =  response["choices"][0]["text"]

    # Combine user_prompt and system_message to create the prompt
    groundedness_prompt = f"""[INST]{groundedness_rater_system_message}\n
                {'user'}: {user_message_template.format(context=context_for_query, question=user_input, answer=answer)}
                [/INST]"""

    # Combine user_prompt and system_message to create the prompt
    relevance_prompt = f"""[INST]{relevance_rater_system_message}\n
                {'user'}: {user_message_template.format(context=context_for_query, question=user_input, answer=answer)}
                [/INST]"""

    response_1 = llm(
            prompt=groundedness_prompt,
            max_tokens=max_tokens,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            stop=['INST'],
            )

    response_2 = llm(
            prompt=relevance_prompt,
            max_tokens=max_tokens,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            stop=['INST'],
            )

    return response_1['choices'][0]['text'],response_2['choices'][0]['text']

### Query 1: What is the protocol for managing sepsis in a critical care unit?

In [74]:
user_input = "What is the protocol for managing sepsis in a critical care unit?"
ground,rel = generate_ground_relevance_response(user_input,max_tokens=350)

print(ground,end="\n\n")
print(rel)

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


 Steps to evaluate the answer:
1. Identify if the answer is derived only from the information presented in the context.
2. Check if the context contains any information related to the protocol for managing sepsis in a critical care unit.
3. Compare the answer with the information available in the context to determine if the answer adheres to the metric.

Explanation:
The answer "I don't know. The context provided does not mention any specific protocol for managing sepsis in a critical care unit" is derived solely from the context as it acknowledges that no such information is present in the context. Therefore, the metric is followed completely.

Rating:
Based on the evaluation criteria, I would rate this answer as 5 (The metric is followed completely).

 Steps to evaluate the context as per the relevance metric:
1. Identify the main aspects of the question, which are "managing sepsis in a critical care unit" and "protocol."
2. Search for information related to these aspects in the cont

### Query 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

In [75]:
user_input2 = "What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?"
ground,rel = generate_ground_relevance_response(user_input2,max_tokens=350)

print(ground,end="\n\n")
print(rel)

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


 Steps to evaluate the answer:
1. Identify the information in the context related to appendicitis symptoms and treatment.
2. Check if the answer only includes the information from the context.
3. Evaluate the extent to which the answer adheres to the metric.

The answer "Common symptoms for appendicitis include pain in the lower right abdomen, loss of appetite, nausea, vomiting, fever, and a feeling of being sick. Appendicitis cannot be cured via medicine alone, and treatment involves surgical removal of the appendix through open or laparoscopic appendectomy" adheres to the metric as it is derived only from the information presented in the context. The symptoms listed are directly mentioned in the context, and the statement that treatment involves surgery is also stated in the context. Therefore, the answer follows the metric completely.

Rating: 5 (The metric is followed completely)

 Steps to evaluate the context as per the relevance metric:
1. Identify the main aspects of the questi

### Query 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

In [70]:
user_input3 = "What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?"
ground,rel = generate_ground_relevance_response(user_input3,max_tokens=350)

print(ground,end="\n\n")
print(rel)

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


 Steps to evaluate the answer:
1. Identify the information directly related to the question in the context.
2. Determine if the AI generated answer only uses information from the identified parts of the context.
3. Evaluate the extent to which the answer adheres to the metric.

Explanation:
The question asks about effective treatments and possible causes for sudden patchy hair loss, specifically mentioning alopecia areata as a potential condition. The context provides information on alopecia areata, its causes, and treatment options.

The AI generated answer mentions that alopecia areata is a condition characterized by sudden patchy hair loss, and the cause is believed to be an autoimmune disorder affecting genetically susceptible people exposed to unclear environmental triggers. It also states that treatment options include corticosteroids, retinoids, or immunosuppressants.

The answer only uses information directly related to alopecia areata and its causes and treatments from the con

### Query 4: What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [71]:
user_input4 = "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?"
ground,rel = generate_ground_relevance_response(user_input4,max_tokens=350)

print(ground,end="\n\n")
print(rel)

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


 Steps to evaluate the answer:
1. Identify the key information in the context related to treatments for traumatic brain injury (TBI).
2. Compare the identified information with the AI generated answer to check if the answer is derived only from the context.
3. Evaluate the extent to which the metric is followed.

Explanation:
The context provides detailed information about the initial treatment for a person with TBI, including ensuring a reliable airway, maintaining adequate ventilation, oxygenation, and blood pressure, and the possible need for surgery in more severe cases. The AI generated answer matches this information exactly, stating that the initial treatment includes these steps and possibly surgery for more severe injuries. Therefore, the metric is followed completely as the answer is derived solely from the context.

Rating:
5 - The metric is followed completely.

 Steps to evaluate the context as per the relevance metric:
1. Identify the main aspects of the question: The que

### Query 5: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

In [72]:
user_input5 = "What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?"
ground,rel = generate_ground_relevance_response(user_input5,max_tokens=350)

print(ground,end="\n\n")
print(rel)

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


 Steps to evaluate the answer:
1. Identify the information in the context that relates to the necessary precautions and treatment steps for a person with a fractured leg.
2. Determine if the AI generated answer includes all the necessary precautions and treatment steps identified in step 1.
3. Check if the information in the AI generated answer is derived only from the context provided.

The AI generated answer adheres to the metric as it includes all the necessary precautions and treatment steps for a person with a fractured leg, which are clearly stated in the context. The answer is also derived directly from the context without introducing any new information or assumptions. Therefore, the metric is followed completely.

Rating: 5 (The metric is followed completely)

 Steps to evaluate the context as per the relevance metric:
1. Identify the main aspects of the question: necessary precautions and treatment steps for a person with a fractured leg, and what should be considered for th

## Actionable Insights and Business Recommendations

#Insights:

Maximum token has played a critical role in completness of the response as the response were non creative and focused on facts. Even in case of smaller queries a lower token size has incomplete answer.

Vector database creation time increases with the number of pages in the PDF document.

Retrieval parameter k is critical as the answer can be spread across multiple contexts. Lesser value can effect the roundness of the answer.

chunk_overlap ensures coherence, especially when context spans across chunks.

Temperature setting should be kept to minimum as the context is medical analysis of various conditions, sysmptoms and appropriate treatment.

#Business Recommendation:

####Contexual relevance:
Experimented with contextual size and with not much difference in text, selected k=3. For the relevance of medical information it's important that continuity of text will not suffer due to lower context size.

####Maintain High groundness:

The model achieved a full score in groundness due to strict prompting. Medical condition not present in the contextual text is not answered as specified in the prompt.

####Optimize Embeddings for domain specific accuracy:

By specifying context specific system messages the results can be refined based on the data provided.

####Continuos Knowledge update:

Picking the latest model is necessary to include the latest medical research and guidlines ensuring the model remains accurate and relevant.






